In [ ]:
from PIL import Image
from IPython.display import display
import sys
import os
from math import pow
sys.path.append("../build/bindings/python/")
from pyfluxrt import *

In [ ]:
scene_path = "../scenes/toml/"
os.environ["SCENE_PATH"] = scene_path
#scene_file = scene_path + "transform/transform_rotate_cube.toml"
#scene_file = scene_path + "ornell/cornell_pieta.toml"
scene_file = scene_path + "sphere.toml"
#scene_file = scene_path + "disklight/disk-light-casts-shadow.toml"
#scene_file = scene_path + "disklight/suzanne-smooth-mirror-disk-light.toml"
#scene_file = scene_path + "disklight/suzanne-smooth-refract-disk-light.toml"
scene = Scene()
loadSceneFromFile(scene, scene_file)
#scene.print()

In [ ]:
# TEMP Shrink scene so the notebook doesn't hang a long time
scene.sensor.pixelwidth = scene.sensor.pixelwidth // 4
scene.sensor.pixelheight = scene.sensor.pixelheight // 4
scene.sensor.print() # FIXME - no output

In [ ]:
renderer = Renderer()
renderer.printConfiguration() # FIXME - no output

In [ ]:
num_samples = 20

In [ ]:
image = Image.new("RGB", (scene.sensor.pixelwidth, scene.sensor.pixelheight))

def renderPixel(x, y, thread_index):
    pixelCenter = vec2(x, y) + vec2(0.5, 0.5)
    standardPixel = scene.sensor.pixelStandardImageLocation(pixelCenter)
    accumRadiance = RadianceRGB()
    for sample_index in range(0, num_samples):
        blurCoord = vec2(0.0, 0.0)
        ray = scene.camera.rayThroughStandardImagePlane(standardPixel, blurCoord)
        intersection = RayIntersection()
        pixelRadiance = RadianceRGB()
        rng = RNG()
        hit = renderer.traceCameraRay(scene, rng, ray, 0.0, 1,
                                    [ VaccuumMedium ], intersection, pixelRadiance)
        if hit:
            accumRadiance += pixelRadiance
    accumRadiance /= num_samples
    # Apply gamma and convert to [0,255] range
    rgb = tuple(int(pow(c, 1.0 / 2.4) * 255)
                for c in (accumRadiance.r, accumRadiance.g, accumRadiance.b))
    image.putpixel((x, y), rgb)

scene.sensor.forEachPixel(renderPixel)
display(image)

In [ ]:
zoom_factor = 4
zoomed_size = (zoom_factor * scene.sensor.pixelwidth, zoom_factor * scene.sensor.pixelheight)
zoomed = image.resize(zoomed_size, Image.BOX)
display(zoomed)